In [27]:
%load_ext autoreload
%autoreload 2

import torch
import torch.nn as nn
import optuna
from modules import Trainer
from modules.competition_dataset import EEGDataset
from modules.utils import evaluate_model
import random
import numpy as np
from torch.utils.data import ConcatDataset, random_split, DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


device(type='cuda')

In [28]:
# from google.colab import drive
# drive.mount('/content/drive')
# data_path = '/content/drive/MyDrive/ai_data/eeg_detection/data/mtcaic3'
# model_path = '/content/drive/MyDrive/ai_data/eeg_detection/checkpoints/ssvep/models/ssvep.pth'
# optuna_db_path = '/content/drive/MyDrive/ai_data/eeg_detection/checkpoints/ssvep/optuna/optuna_studies.db'
data_path = './data/mtcaic3'
model_path = './checkpoints/mi/models/the_honored_one.pth'
optuna_db_path = './checkpoints/mi/optuna/the_honored_one.db'
eeg_channels = ["C3", "PZ", "OZ"]

In [29]:
# Add this at the beginning of your notebook, after imports
def set_random_seeds(seed=42):
    """Set random seeds for reproducibility"""

    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

# Call this function before creating datasets and models
set_random_seeds(42)

In [30]:
window_length = 128 * 2 # ensure divisble by 64 the kernel size
stride = window_length // 3
batch_size = 64

dataset_train = EEGDataset(
    data_path,
    window_length=window_length,
    stride=stride,
    domain="time",
    data_fraction=1,
    hardcoded_mean=True,
    task="mi",
    eeg_channels=eeg_channels,
)

dataset_val = EEGDataset(
    data_path=data_path,
    window_length=window_length,
    stride=stride,
    split='validation',
    read_labels=True,
    hardcoded_mean=True,
    data_fraction=1,
    task="mi",
    eeg_channels=eeg_channels,
)

combined = ConcatDataset([dataset_train, dataset_val])
train_len = int(len(combined) * 0.8)
val_len = len(combined) - train_len
train_ds, val_ds = random_split(combined, [train_len, val_len])

train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_ds,   batch_size=batch_size)

task: mi, split: train, domain: time, data_fraction: 1
skipped: 0/2400
data shape: (42127, 3, 256), mean shape: (1, 3, 1)
task: mi, split: validation, domain: time, data_fraction: 1
skipped: 0/50
data shape: (857, 3, 256), mean shape: (1, 3, 1)


In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
        super(LSTMModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim
        self.lstm = nn.LSTM(input_dim, hidden_dim, layer_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x, h0=None, c0=None):
        if h0 is None or c0 is None:
            h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).to(x.device)
            c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).to(x.device)

        out, (hn, cn) = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

class DepthWiseConv2D(nn.Module):
    def __init__(self, in_channels, kernel_size, dim_mult=1, padding=0, bias=False):
        super(DepthWiseConv2D, self).__init__()
        self.depthwise = nn.Conv2d(in_channels, in_channels * dim_mult, padding=padding, kernel_size=kernel_size, groups=in_channels, bias=bias)

    def forward(self, x: torch.Tensor):
        return self.depthwise(x)


class SeperableConv2D(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, padding, bias=False):
        super(SeperableConv2D, self).__init__()
        self.depthwise = DepthWiseConv2D(in_channels, kernel_size, padding=padding)
        self.pointwise = nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=bias)

    def forward(self, x):
        out = self.depthwise(x)
        out = self.pointwise(out)
        return out

class SSVEPClassifier(nn.Module):
    # EEG Net Based
    # todo look at this https://paperswithcode.com/paper/a-transformer-based-deep-neural-network-model
    def __init__(self, n_electrodes=16, out_dim=4, dropout=0.25, kernLength=256, F1=96, D=1, F2=96, hidden_dim=100, layer_dim=1):
        super().__init__()

        # B x C x T
        self.block_1 = nn.Sequential(
            nn.Conv2d(1, F1, (1, kernLength), padding='same', bias=False),
            nn.BatchNorm2d(F1),
            #
            DepthWiseConv2D(F1, (n_electrodes, 1), dim_mult=D, bias=False),
            nn.BatchNorm2d(F1*D),
            nn.ELU(),
            nn.MaxPool2d((1, 2)), # todo try making this max pool
            nn.Dropout(dropout),
            #
            SeperableConv2D(F1 * D, F2, kernel_size=(1, 16), padding='same', bias=False),
            nn.BatchNorm2d(F2),
            nn.ELU(),
            nn.MaxPool2d((1, 2)),
            nn.Dropout(dropout),
        )

        self.lstm_head = LSTMModel(F2, hidden_dim, layer_dim, out_dim)

    def forward(self, x: torch.Tensor):
        """expected input shape: BxCxT"""
        x = x.unsqueeze(1)
        y = self.block_1(x) # B x F1 x 1 x time_sub

        y = y.squeeze(2) # B x F1 x time_sub
        y = y.permute(0, 2, 1) # B x time_sub x F1
        y = self.lstm_head(y)

        return y

dummy_x = torch.randn(5, 3, 256).to(device)
model = SSVEPClassifier(
    dropout=0.26211635308091535,
    n_electrodes=3,
    out_dim=2,
    kernLength=256,
    F1 = 32,
    D = 3,
    F2 = 32,
    hidden_dim=256,
    layer_dim=3,
).to(device)

model(dummy_x)

tensor([[1.5133e-03, 5.1941e-02],
        [3.7769e-03, 5.3241e-02],
        [2.6249e-04, 5.2329e-02],
        [2.4431e-03, 5.2711e-02],
        [2.7204e-06, 5.1845e-02]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [33]:
try:
    model.load_state_dict(torch.load(model_path, weights_only=True))
except Exception:
    print("skipping model loading...")


opt = torch.optim.Adam(model.parameters(), lr=0.0003746351873334935)
criterion = nn.CrossEntropyLoss()
avg_losses = []
val_accuracies = []
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    opt,
    mode='min',        # “min” if you want to reduce lr when the quantity monitored has stopped decreasing
    factor=0.5,        # new_lr = lr * factor
    patience=20,        # number of epochs with no improvement after which lr will be reduced
    threshold=1e-4,    # threshold for measuring the new optimum, to only focus on significant changes
    threshold_mode='rel', # `'rel'` means compare change relative to best value. Could use `'abs'`.
    cooldown=0,        # epochs to wait before resuming normal operation after lr has been reduced
    min_lr=1e-6,       # lower bound on the lr
)

epochs = 4000
for epoch in range(epochs):
    avg_loss = 0
    model.train()
    for x, y in train_loader:
        x = x.to(device)
        y = y.to(device).to(torch.int64)
        y_pred = model(x).to(device)

        loss = criterion(y_pred, y)
        opt.zero_grad()
        loss.backward()
        opt.step()
        avg_loss += loss.item()

    avg_loss /= len(train_loader)
    avg_losses.append(avg_loss)
    scheduler.step(avg_loss)

    if epoch % 5 == 0:
        evaluation = evaluate_model(model, val_loader, device)
        val_accuracies.append(evaluation)
        model.cpu()
        torch.save(model.state_dict(), model_path)
        model.to(device)
        print(f"epoch: {epoch}, avg_loss: {avg_loss}, val_evaluation: {evaluation}, lr: {opt.param_groups[0]['lr']}")

epoch: 0, avg_loss: 0.22998837412068393, val_evaluation: 0.845178550657206, lr: 0.0003746351873334935
epoch: 5, avg_loss: 0.23133591101349066, val_evaluation: 0.8406420844480633, lr: 0.0003746351873334935
epoch: 10, avg_loss: 0.22475356866214355, val_evaluation: 0.8471559846458067, lr: 0.0003746351873334935
epoch: 15, avg_loss: 0.22771020912215612, val_evaluation: 0.8440153541933233, lr: 0.0003746351873334935
epoch: 20, avg_loss: 0.2190342526675157, val_evaluation: 0.8384320111666861, lr: 0.0003746351873334935
epoch: 25, avg_loss: 0.2175467263588675, val_evaluation: 0.8394788879841806, lr: 0.0003746351873334935
epoch: 30, avg_loss: 0.21255209111369675, val_evaluation: 0.8341281842503199, lr: 0.0003746351873334935
epoch: 35, avg_loss: 0.20875239453960528, val_evaluation: 0.8433174363149936, lr: 0.0003746351873334935
epoch: 40, avg_loss: 0.21021467909826222, val_evaluation: 0.8418052809119461, lr: 0.0003746351873334935
epoch: 45, avg_loss: 0.20361291338796952, val_evaluation: 0.839711527

KeyboardInterrupt: 

In [25]:
class CustomTrainer(Trainer):
    # This method is called by _objective during an Optuna trial
    def prepare_trial_run(self):
        assert isinstance(self.trial, optuna.Trial), "Trial not set!"

        # 1. Define Hyperparameters for this trial
        #    a. Data/Loader parameters
        window_length = self.trial.suggest_categorical("window_length", [128, 256, 640]) # e.g. 64*2, 64*4, 64*10
        batch_size = self.trial.suggest_categorical("batch_size", [32, 64])

        #    b. Model architecture parameters
        kernLength = self.trial.suggest_categorical("kernLength", [64, 128, 256])
        F1 = self.trial.suggest_categorical("F1", [8, 16, 32])
        D = self.trial.suggest_categorical("D", [1, 2, 3])
        F2 = self.trial.suggest_categorical("F2", [16, 32, 64]) # F2 must be F1 * D
        hidden_dim = self.trial.suggest_categorical("hidden_dim", [64, 128, 256])
        layer_dim = self.trial.suggest_categorical("layer_dim", [1, 2, 3])
        dropout = self.trial.suggest_float("dropout", 0.1, 0.6)
        
        #    c. Optimizer parameters
        lr = self.trial.suggest_float("lr", 1e-4, 1e-2, log=True)

        # 2. Prepare the data using these parameters
        super()._prepare_data(is_trial=True, batch_size=batch_size, window_length=window_length)
        
        assert self.dataset is not None, "Dataset was not created correctly"
        n_electrodes = self.dataset.datasets[0].data[0].shape[0] # Get shape from underlying dataset

        # 3. Build the model and optimizer
        self.model = SSVEPClassifier(
            n_electrodes=n_electrodes, # Use value from data
            dropout=dropout,
            kernLength=kernLength,
            F1=F1,
            D=D,
            F2=F1 * D, # F2 is dependent on F1 and D
            hidden_dim=hidden_dim,
            layer_dim=layer_dim,
        ).to(self.device)
        
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=lr)

    # This method is called by train() for the final run
    def prepare_final_run(self):
        # 1. Get the best hyperparameters from the completed study
        study = self._get_study()
        best_params = study.best_params
        
        # 2. Prepare data using the best params
        super()._prepare_data(is_trial=False) # is_trial=False handles getting params from study
        
        assert self.dataset is not None, "Dataset was not created correctly"
        n_electrodes = self.dataset.datasets[0].data[0].shape[0]

        # 3. Build the final model and optimizer
        self.model = SSVEPClassifier(
            n_electrodes=n_electrodes,
            out_dim=2,
            dropout=best_params["dropout"],
            kernLength=best_params["kernLength"],
            F1=best_params["F1"],
            D=best_params["D"],
            F2=best_params["F1"] * best_params["D"],
            hidden_dim=best_params["hidden_dim"],
            layer_dim=best_params["layer_dim"],
        ).to(self.device)
        
        # Optional: Load pre-existing weights if you are resuming
        try:
            self.model.load_state_dict(torch.load(self.model_path))
            print(f"Loaded existing model weights from {self.model_path}")
        except Exception:
            print(f"No existing model weights found at {self.model_path}. Training from scratch.")
        
        lr = 0.00018182233882257615 # best_params["lr"]
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr)
        self.scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            self.optimizer,
            mode='max',        # “min” if you want to reduce lr when the quantity monitored has stopped decreasing
            factor=0.5,        # new_lr = lr * factor
            patience=20,        # number of epochs with no improvement after which lr will be reduced
            threshold=1e-4,    # threshold for measuring the new optimum, to only focus on significant changes
            threshold_mode='rel', # `'rel'` means compare change relative to best value. Could use `'abs'`.
            cooldown=0,        # epochs to wait before resuming normal operation after lr has been reduced
            min_lr=1e-6,       # lower bound on the lr
        )

trainer = CustomTrainer(
        data_path=data_path,
        optuna_db_path=optuna_db_path,
        model_path=model_path,
        train_epochs=500, # Final training epochs
        tune_epochs=50,   # Epochs per trial
        optuna_n_trials=50,
        task="mi",
        eeg_channels=eeg_channels,
        data_fraction=0.4
    )

In [26]:
delete_existing = False
trainer.optimize(delete_existing)

[I 2025-06-24 13:08:57,087] Using an existing study with name 'mi_classifier_optimization' instead of creating a new one.


task: mi, split: train, domain: time, data_fraction: 0.4
Using 40.0% of data: 960/960 samples
skipped: 0/960
data shape: (16860, 3, 256), mean shape: (1, 3, 1)
task: mi, split: validation, domain: time, data_fraction: 1.0
skipped: 0/50
data shape: (857, 3, 256), mean shape: (1, 3, 1)
Data prepared: Train batches=222, Val batches=56
Epoch 0/50, Validation Accuracy: 0.5025, Avg Loss: 0.7742, lr: 0.0006426703883202322
Epoch 1/50, Validation Accuracy: 0.5240, Avg Loss: 0.6915, lr: 0.0006426703883202322
Epoch 2/50, Validation Accuracy: 0.5025, Avg Loss: 0.6897, lr: 0.0006426703883202322
Epoch 3/50, Validation Accuracy: 0.5025, Avg Loss: 0.6895, lr: 0.0006426703883202322
Epoch 4/50, Validation Accuracy: 0.5257, Avg Loss: 0.6890, lr: 0.0006426703883202322
Epoch 5/50, Validation Accuracy: 0.5260, Avg Loss: 0.6877, lr: 0.0006426703883202322
Epoch 6/50, Validation Accuracy: 0.5274, Avg Loss: 0.6887, lr: 0.0006426703883202322
Epoch 7/50, Validation Accuracy: 0.5102, Avg Loss: 0.6883, lr: 0.000642

[I 2025-06-24 13:10:29,169] Trial 2 finished with value: 0.618510158013544 and parameters: {'window_length': 256, 'batch_size': 64, 'kernLength': 256, 'F1': 8, 'D': 2, 'F2': 16, 'hidden_dim': 64, 'layer_dim': 1, 'dropout': 0.40305525348217675, 'lr': 0.0006426703883202322}. Best is trial 2 with value: 0.618510158013544.


task: mi, split: train, domain: time, data_fraction: 0.4
Using 40.0% of data: 960/960 samples
skipped: 0/960
data shape: (16860, 3, 256), mean shape: (1, 3, 1)
task: mi, split: validation, domain: time, data_fraction: 1.0
skipped: 0/50
data shape: (857, 3, 256), mean shape: (1, 3, 1)
Data prepared: Train batches=222, Val batches=56
Epoch 0/50, Validation Accuracy: 0.5169, Avg Loss: 0.7611, lr: 0.0001965353092543613
Epoch 1/50, Validation Accuracy: 0.5169, Avg Loss: 0.6929, lr: 0.0001965353092543613
Epoch 2/50, Validation Accuracy: 0.5274, Avg Loss: 0.6912, lr: 0.0001965353092543613
Epoch 3/50, Validation Accuracy: 0.5169, Avg Loss: 0.6904, lr: 0.0001965353092543613
Epoch 4/50, Validation Accuracy: 0.5265, Avg Loss: 0.6901, lr: 0.0001965353092543613
Epoch 5/50, Validation Accuracy: 0.5231, Avg Loss: 0.6886, lr: 0.0001965353092543613
Epoch 6/50, Validation Accuracy: 0.5347, Avg Loss: 0.6899, lr: 0.0001965353092543613
Epoch 7/50, Validation Accuracy: 0.5234, Avg Loss: 0.6886, lr: 0.000196

[I 2025-06-24 13:13:05,916] Trial 3 finished with value: 0.6159706546275395 and parameters: {'window_length': 256, 'batch_size': 64, 'kernLength': 64, 'F1': 8, 'D': 3, 'F2': 64, 'hidden_dim': 256, 'layer_dim': 1, 'dropout': 0.3501663697670412, 'lr': 0.0001965353092543613}. Best is trial 2 with value: 0.618510158013544.


...

[I 2025-06-24 14:01:48,965] Trial 50 pruned. 


task: mi, split: train, domain: time, data_fraction: 0.4
Using 40.0% of data: 960/960 samples
skipped: 0/960
data shape: (36600, 3, 128), mean shape: (1, 3, 1)
task: mi, split: validation, domain: time, data_fraction: 1.0
skipped: 0/50
data shape: (1862, 3, 128), mean shape: (1, 3, 1)
Data prepared: Train batches=481, Val batches=121


[I 2025-06-24 14:02:08,072] Trial 51 pruned. 



--- Optimization Finished ---
Study statistics: 
  Number of finished trials: 52
  Number of pruned trials: 40
  Number of complete trials: 10
Best trial:
  Value: 0.6703496685298324
  Best hyperparameters: 
    window_length: 128
    batch_size: 64
    kernLength: 256
    F1: 32
    D: 2
    F2: 16
    hidden_dim: 64
    layer_dim: 1
    dropout: 0.26679832268110276
    lr: 0.0010570001537933554


{'window_length': 128,
 'batch_size': 64,
 'kernLength': 256,
 'F1': 32,
 'D': 2,
 'F2': 16,
 'hidden_dim': 64,
 'layer_dim': 1,
 'dropout': 0.26679832268110276,
 'lr': 0.0010570001537933554}

In [ ]:
trainer.train()

In [ ]:
trainer._prepare_training(False)
trainer.model.eval()
f"test accuracy: {evaluate_model(trainer.model, trainer.eval_loader, device)}"